# Model Inference

Muhammad Daffa  
FTDS   
BSD - 007

NLP - Review App Sentiment Analysis

**Inference Prediction Review Sentiment from Customer (Good Review / Bad Review)**

---

# Import Libraries

In [1]:
import pandas as pd # Import neccesary Library
import numpy as np
from tensorflow.keras.models import load_model
import contractions

from nltk.stem import WordNetLemmatizer
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

---

# Import Model

In [2]:
# Path to the downloaded model file
model_path = '/Users/duhammaddaffa/Downloads/model_final'

# Load the model
model = load_model(model_path)

---

# Text Preprocessing

Applying text preprocessing from the model

In [3]:
# Def Function to expand contractions
def expand_contractions(text):
    return contractions.fix(text) # From Contractions library

In [4]:
# Define Lemmatizer
lemmatizer = WordNetLemmatizer()

# Define Stopwords
stopword = set(stopwords.words('english'))
stopword.remove('not') # We dont want to remove 'not' since we already did contractions expanding

# Function for text processing
def text_preprocessing(text):
  '''
  Function to preprocess text including case folding, mention removal, hashtag removal,
  newline removal, whitespace removal, url removal, non-letter removal, tokenization,
  stopword removal, and lemmatization
  '''
  # Case folding
  text = text.lower()

  # Mention removal
  text = re.sub("@[A-Za-z0-9_]+", " ", text)

  # Hashtags removal
  text = re.sub("#[A-Za-z0-9_]+", " ", text)

  # Newline removal (\n)
  text = re.sub(r"\\n", " ",text)

  # Whitespace removal
  text = text.strip()

  # Remove non-latin words from the sentences. There are some other language (chinese) in the reviews
  text = re.sub(r'[^\x00-\x7f]', r'', text)

  # URL removal
  text = re.sub(r"http\S+", " ", text)
  text = re.sub(r"www.\S+", " ", text)

  # Non-letter removal (such as emoticon, symbol (like μ, $, 兀), etc
  text = re.sub("[^A-Za-z\s']", " ", text)

  # Tokenization
  tokens = word_tokenize(text)

  # Stopwords removal
  tokens = [word for word in tokens if word not in stopword]

  # Removing Words less than 2 lengths
  tokens =  [word for word in tokens if len(word) > 2]

 # Lemmatization
  tokens = [lemmatizer.lemmatize(word) for word in tokens]

  # Combining Tokens
  text = ' '.join(tokens)

  return text

---

## Inference New Data

In [5]:
# Create New Data 

inference = {
    'Review' : 'Sucks apps always error and too many ads'                                
}

inference = pd.DataFrame([inference])
inference

,Review
0,Sucks apps always error and too many ads


### Preprocessing the Inference Data

In [6]:
# Apply the contraction expansion to review column
inference['text_processed'] = inference['Review'].apply(expand_contractions)

# Apply the test processing function
inference['text_processed'] = inference['text_processed'].apply(lambda x: text_preprocessing(x))
inference

,Review,text_processed
0,Sucks apps always error and too many ads,suck apps always error many ad


---

In [7]:
prediction = np.argmax(model.predict(inference['text_processed']), axis=1)

1/1 [==============================] - 1s 516ms/step


In [8]:
if prediction[0] == 0:
    print(f'Review Sentiment is Considered as BAD REVIEW')

else:
    print(f'Review Sentiment is Considered as GOOD REVIEW')

Review Sentiment is Considered as BAD REVIEW
